In [1]:
from functools import reduce
from itertools import permutations
import random

from cube import Cube
from alt_map import create_hex_map, valid_cubes
from area import Area
from voronoi import simple_voronoi, voronoi, iterative_voronoi

In [2]:
random.seed(1945)
n_x = 129
n_y = 65
rgb_from_ijk = {cub.tuple():(random.randint(0,255), random.randint(0,255), random.randint(0,255)) for cub in valid_cubes(n_x,n_y)}

In [3]:
max_x = 10*(n_x*3-3)
max_y = 17*(n_y*2-2)

In [4]:
img = create_hex_map(rgb_from_ijk, max_x,max_y,n_x,n_y)
img.show()

In [5]:
pid_from_cube = {cub.tuple(): ind for ind, cub in enumerate(valid_cubes(n_x,n_y))}
cube_from_pid = {v: k for k, v in pid_from_cube.items()}
weight_from_pid = {pid: random.randint(1,8) for pid in cube_from_pid.keys()}

In [6]:
weight_from_cube = {Cube(cube_from_pid[k]): v for k,v in weight_from_pid.items()}

In [7]:
centers = random.sample(sorted(cube_from_pid.values()),40)
print(centers)
print(sum([sum([x == c for x in centers]) for c in centers]))

[(112, -78, -34), (29, -70, 41), (36, -53, 17), (25, -72, 47), (85, -90, 5), (94, -106, 12), (106, -110, 4), (38, -45, 7), (126, -116, -10), (64, -62, -2), (93, -79, -14), (31, -45, 14), (123, -123, 0), (26, -76, 50), (2, -40, 38), (98, -83, -15), (35, -49, 14), (82, -96, 14), (30, -72, 42), (69, -38, -31), (106, -113, 7), (47, -57, 10), (79, -95, 16), (124, -69, -55), (53, -80, 27), (26, -73, 47), (97, -92, -5), (15, -70, 55), (115, -83, -32), (20, -27, 7), (67, -93, 26), (33, -30, -3), (115, -73, -42), (42, -40, -2), (72, -57, -15), (43, -46, 3), (12, -53, 41), (116, -116, 0), (5, -37, 32), (73, -40, -33)]
40


In [8]:
centers, result, distmap = iterative_voronoi(num_centers=80, weight_from_cube=weight_from_cube, min_size=61)
img = create_hex_map({cub.tuple(): rgb_from_ijk[centers[v].tuple()] for cub, v in result.items()}, max_x,max_y,n_x,n_y)
img.show()
img.save("weighted.png")

59
69
72
72


In [9]:
cids = sorted(set(result.values()))
chunks = []
for cid in cids:
    chunks.append(Area(cid, [k for k, v in result.items() if v == cid]))
for chunk in chunks:
    chunk.calc_edges(result)

In [10]:
triangles = set()
for cid in cids:
    if chunks[cid].outside:
        continue
    for oid1 in chunks[cid].self_edges.keys():
        if chunks[oid1].outside:
            continue
        for oid2 in chunks[oid1].self_edges.keys():
            if chunks[oid2].outside:
                continue
            if cid in chunks[oid2].self_edges.keys():
                # Found a triangle!
                short_edge = min(
                    len(chunks[cid].self_edges[oid1]),  len(chunks[cid].other_edges[oid1]),
                    len(chunks[cid].self_edges[oid2]),  len(chunks[cid].other_edges[oid2]),
                    len(chunks[oid1].self_edges[oid2]), len(chunks[oid1].other_edges[oid2]),
                )
                triangles.add((tuple(sorted([cid,oid1,oid2])), short_edge, sum([len(chunks[x].members) for x in [cid,oid1,oid2]])))

In [11]:
print(len(triangles))
sorted(triangles, key=lambda x: x[1], reverse=True)[:10]

71


[((31, 53, 59), 7, 343),
 ((14, 25, 38), 7, 430),
 ((38, 52, 74), 7, 320),
 ((1, 14, 24), 7, 393),
 ((14, 24, 38), 7, 324),
 ((12, 37, 52), 7, 321),
 ((46, 49, 74), 6, 287),
 ((7, 11, 42), 6, 314),
 ((47, 61, 62), 6, 291),
 ((14, 26, 30), 6, 320)]

In [12]:
quads = set()
for tri in triangles:
    for other in triangles:
        ids = set(tri[0]).union(other[0])
        if len(ids) == 4:
            quads.add((tuple(sorted(ids)), min(tri[1], other[1]), sum([len(chunks[x].members) for x in ids])))


In [13]:
print(len(quads))
sorted(quads, key=lambda x: x[1], reverse=True)[:10]

93


[((14, 24, 25, 38), 7, 468),
 ((1, 14, 24, 38), 7, 509),
 ((1, 47, 61, 62), 5, 476),
 ((31, 53, 59, 71), 5, 431),
 ((27, 53, 66, 71), 5, 420),
 ((31, 53, 66, 71), 5, 399),
 ((38, 46, 52, 74), 4, 438),
 ((53, 59, 66, 76), 4, 419),
 ((1, 29, 61, 62), 4, 479),
 ((7, 11, 25, 42), 4, 458)]

In [14]:
abc_ids = sorted(triangles, key=lambda x: x[1], reverse=True)[0]
print(abc_ids)


((31, 53, 59), 7, 343)


In [15]:
a,b,c = abc_ids[0]

In [16]:
abc_center = list(chunks[a].self_edges[b].intersection(chunks[a].self_edges[c]))[0]

In [17]:
chunks[a].self_edges[c]

{Cube(111, -96, -15),
 Cube(112, -100, -12),
 Cube(112, -99, -13),
 Cube(112, -97, -15),
 Cube(113, -99, -14),
 Cube(113, -98, -15),
 Cube(113, -101, -12),
 Cube(114, -102, -12),
 Cube(114, -103, -11)}

In [18]:
img = create_hex_map({cub.tuple(): rgb_from_ijk[centers[v].tuple()] if v in abc_ids[0] else (0,0,0) for cub, v in result.items()}, max_x,max_y,n_x,n_y)
img.show()

In [39]:
CENTER_SIZE_LIST = [7,5,5,5]
KINGDOM_SIZE_LIST = [[6,4,4,4,4], [4,4,3,3], [4,4,3,3], [4,4,3]]
BORDER_SIZE_LIST = [4,4,4]

In [ ]:
sum(CENTER_SIZE_LIST)+3*61+6*sum(BORDER_SIZE_LIST)

277

In [19]:
# Basic approach here:
# hex that borders all three kingdoms should be the middle of a seven-hex. At least one of them should be--maybe our first guess is wrong.
# Make that the central duchy of abc. Then we have to get 3 size-5 duchies; might as well take one from a, b, and c.
# Now we need ab/bc/ac; those can start from the edge zones and grow outward from there.

# let's calculate the center-dist the dumb way!
_, _, cdistmap = voronoi([abc_center],weight_from_cube)

In [65]:
max([cdistmap[k] for k,v in result.items() if v in abc_ids[0]])

53

In [20]:
img = create_hex_map({cub.tuple(): (cdistmap[cub]*4,0,0) if v in abc_ids[0] else (0,0,0) for cub, v in result.items()}, max_x,max_y,n_x,n_y)
img.show()

In [21]:
_, _, adistmap = voronoi(list(chunks[a].self_edges[b].union(chunks[a].self_edges[c])),weight_from_cube)
img = create_hex_map({cub.tuple(): (min(255,adistmap[cub]*4),0,0) if v == a else (0,0,0) for cub, v in result.items()}, max_x,max_y,n_x,n_y)
img.show()

In [22]:
cchunks = []  # These are the county-level regions. There are 4 of them in the center duchy, 6 of them in each border duchy (or a pair of 3, still have to think about that), and 16 in each kingdom.
# That's 4+6*3+16*3=70 cchunks.
# The first one is always the center plus its neighbors. TODO: check to make sure these are always inside, tho I don't think it'll matter for the ones we expect to grab.
cchunks.append(Area(0,set([abc_center]).union(abc_center.neighbors())))

In [23]:
# now the center duchy has three more counties, each of which I think we should grow out of the a/b/c regions.
for cid, other in enumerate([a,b,c]):
    options = {k:weight_from_cube[k] for k in cdistmap.keys() if k in chunks[other].members and k not in cchunks[0].members}
    _, _, selection = voronoi([min(options, key=cdistmap.get)],options)
    ss = sorted(selection, key=selection.get)
    cchunks.append(Area(cid,ss[:5]))

In [26]:
ccrgb = {k.tuple():(cdistmap[k]*4,0,0) if v in abc_ids[0] else (0,0,0) for k,v in result.items()}
for cind, cc in enumerate(cchunks):
    for k in cc.members:
        ccrgb[k.tuple()] = (0,255-3*cind,3*cind)
img = create_hex_map(ccrgb, max_x,max_y,n_x,n_y)
img.show()

In [30]:
allocated = reduce(lambda x,y: x.union(y), [cc.members for cc in cchunks])
new_centers = []
for o1, o2 in [(a,b),(b,c),(a,c)]:
    options = {k for k in chunks[o1].self_edges[o2].union(chunks[o1].other_edges[o2]) if k not in allocated}
    new_centers.append(min(options, key=cdistmap.get))
new_centers

[Cube(110, -94, -16), Cube(107, -95, -12), Cube(111, -98, -13)]

In [37]:
dchunks = []  # These are the duchy-level regions. There is 1 in the center duchy, 2 of them in each border duchy, and 4 in each kingdom.
# That's 1+2*3+4*3=19 dchunks.
dchunks.append(Area(0,allocated))

In [103]:
subweights = {k:weight_from_cube[k] for k,v in result.items() if k not in allocated and v in abc_ids[0]}
new_centers, ccsplit, ccdistmap = voronoi(new_centers,subweights)

In [43]:
ccrgb = {k.tuple():(0,0,0) for k in result}
for cind, cc in enumerate(cchunks):
    for k in cc.members:
        ccrgb[k.tuple()] = (0,255-30*cind,30*cind)
# for k,v in ccsplit.items():
#     if k in new_centers:
#         ccrgb[k.tuple()] = (255,255,255)
#     elif v == 5:
#         ccrgb[k.tuple()] = (0,0,0)
#     else:
#         ccrgb[k.tuple()] = rgb_from_ijk[new_centers[v].tuple()]
img = create_hex_map(ccrgb, max_x,max_y,n_x,n_y)
img.show()

In [45]:
print(len(cchunks))

7


In [109]:
set(ccsplit.values())

{0, 1, 2, 3, 4, 5}

In [111]:
[len([k for k,v in ccsplit.items() if v == x]) for x in sorted(set(ccsplit.values()))]

[180, 62, 61, 27, 23, 3]

In [28]:
borderdistmap = {}
for x,y,z in permutations([a,b,c]):
    _, _, odistmap = voronoi(list(chunks[x].self_edges[y].union(chunks[x].self_edges[z])),{k:v for k,v in weight_from_cube.items() if result[k] == x})
    borderdistmap.update(odistmap)
img = create_hex_map({cub.tuple(): (min(255,borderdistmap[cub]*4),0,0) if v in abc_ids[0] else (0,0,0) for cub, v in result.items()}, max_x,max_y,n_x,n_y)
img.show()

In [27]:
ccrgb = {k.tuple():(weight_from_cube[k]*16,0,0) if v in abc_ids[0] else (0,0,0) for k,v in result.items()}
img = create_hex_map(ccrgb, max_x,max_y,n_x,n_y)
img.show()

In [29]:
img = create_hex_map({cub.tuple(): (min(255,borderdistmap[cub]*4+cdistmap[cub]),0,0) if v in abc_ids[0] else (0,0,0) for cub, v in result.items()}, max_x,max_y,n_x,n_y)
img.show()

In [33]:
subweights = {k:borderdistmap[k]*2+cdistmap[k] for k,v in result.items() if k not in allocated and v in abc_ids[0]}
new_centers, ccsplit, ccdistmap = voronoi(new_centers,subweights)
ccrgb = {k.tuple():(min(255,ccdistmap.get(k,0)),0,0) if v in abc_ids[0] else (0,0,0) for k,v in result.items()}
img = create_hex_map(ccrgb, max_x,max_y,n_x,n_y)
img.show()

In [34]:
ccrgb = {k.tuple():(min(255,ccdistmap.get(k,0)),0,0) if v in abc_ids[0] else (0,0,0) for k,v in result.items()}
img = create_hex_map(ccrgb, max_x,max_y,n_x,n_y)
img.show()

In [ ]:
img = create_hex_map(ccrgb, max_x,max_y,n_x,n_y)
img.show()

In [41]:
cchunks

In [39]:
# each pair of border duchies is just the 24 'closest' hexes determined by our distance function.
for cid, other in enumerate([a,b,c]):
    options = {k:weight_from_cube[k] for k in cdistmap.keys() if k in chunks[other].members and k not in cchunks[0].members}
    _, _, selection = voronoi([min(options, key=cdistmap.get)],options)
    ss = sorted(selection, key=selection.get)
    cchunks.append(Area(cid,ss[:23]))

In [47]:
cchunks[4].members

{Cube(111, -94, -17),
 Cube(112, -94, -18),
 Cube(112, -95, -17),
 Cube(113, -96, -17),
 Cube(113, -95, -18)}